In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
import wandb
from wandb.keras import WandbMetricsLogger

import os
import sys
import datetime

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import log_loss

import warnings; warnings.filterwarnings('ignore')

sys.path.append('..')
from config import CFG
from src.model.include_concat import NN
CFG = CFG()
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 100)

2023-03-20 15:57:19.251853: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
from colorama import Style, Fore
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
mgt = Style.BRIGHT + Fore.MAGENTA
grn = Style.BRIGHT + Fore.GREEN
gld = Style.BRIGHT + Fore.YELLOW
res = Style.RESET_ALL

rc = {
    "axes.facecolor": "#FFFEF8",
    "figure.facecolor": "#FFFEF8",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}
sns.set(rc=rc)
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

In [17]:
def log_(column):
    return np.log(-min(column) + 1 + column)

In [18]:
orig = pd.read_csv(os.path.join(CFG.RAW_DATA, 'Pulsar.csv'))
train = pd.read_csv(os.path.join(CFG.RAW_DATA, 'train.csv')).drop(columns='id')
test = pd.read_csv(os.path.join(CFG.RAW_DATA, 'test.csv')).drop(columns='id')

train[['Skewness', 'Skewness_DMSNR_Curve']] = train.apply({'Skewness': log_, 'Skewness_DMSNR_Curve': log_})
test[['Skewness', 'Skewness_DMSNR_Curve']] = test.apply({'Skewness': log_, 'Skewness_DMSNR_Curve': log_})

In [22]:
X = train.drop('Class', axis=1)
y = train['Class']

In [20]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, mode='min', restore_best_weights=True, verbose=2
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=10, mode='min', restore_best_weights=True, min_lr=1e-12, verbose=2
    ),
    WandbMetricsLogger()
]

In [25]:
ensemble = NN(test)
ensemble.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=CFG.LR),
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)
k_fold = RepeatedStratifiedKFold(n_splits=CFG.NFOLDS, n_repeats=CFG.REPEATS, random_state=CFG.SEED)

params = {
    'folds': CFG.NFOLDS,
    'repeats': CFG.REPEATS,
    'batch_size': CFG.BATCH_SIZE,
    'learning_rate': CFG.LR,
}
wandb.init(project='S3E10', group='NN', name='NN', config=params)

train_log_loss = []
oof_log_loss = []
models = []
for train_idx, val_idx in k_fold.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    history = ensemble.fit(
        X_train, y_train, 
        batch_size=CFG.BATCH_SIZE, 
        epochs=CFG.EPOCHS, callbacks=callbacks, 
        validation_data=(X_val, y_val),
        use_multiprocessing=True
    )

    train_preds = ensemble.predict(X_train)
    train_loss = log_loss(y_train, train_preds)
    train_log_loss.append(train_loss)

    oof_preds = ensemble.predict(X_val)
    oof_loss = log_loss(y_val, oof_preds)
    oof_log_loss.append(oof_loss)

    models.append(ensemble)
wandb.finish()

epoch/binary_accuracy,▁▆▇▇█
epoch/epoch,▁▃▅▆█
epoch/learning_rate,▁▁▁▁▁
epoch/loss,█▃▂▂▁
epoch/lr,▁▁▁▁▁
epoch/val_binary_accuracy,▁▄▅▅█
epoch/val_loss,█▄▃▄▁
epoch/binary_accuracy,0.98825
epoch/epoch,4
epoch/learning_rate,0.001
epoch/loss,0.04384


Epoch 1/50
4899/4899 [==============================] - 49s 8ms/step - loss: 0.0666 - binary_accuracy: 0.9806 - val_loss: 0.0475 - val_binary_accuracy: 0.9885 - lr: 0.0010
Epoch 2/50
4899/4899 [==============================] - 38s 8ms/step - loss: 0.0523 - binary_accuracy: 0.9859 - val_loss: 0.0417 - val_binary_accuracy: 0.9883 - lr: 0.0010
Epoch 3/50
4899/4899 [==============================] - 38s 8ms/step - loss: 0.0482 - binary_accuracy: 0.9868 - val_loss: 0.0385 - val_binary_accuracy: 0.9890 - lr: 0.0010
Epoch 4/50
4899/4899 [==============================] - 38s 8ms/step - loss: 0.0445 - binary_accuracy: 0.9876 - val_loss: 0.0365 - val_binary_accuracy: 0.9904 - lr: 0.0010
Epoch 5/50
4899/4899 [==============================] - 38s 8ms/step - loss: 0.0427 - binary_accuracy: 0.9886 - val_loss: 0.0363 - val_binary_accuracy: 0.9900 - lr: 0.0010
Epoch 6/50
4899/4899 [==============================] - 38s 8ms/step - loss: 0.0427 - binary_accuracy: 0.9887 - val_loss: 0.0357 - val_binar